In [91]:
import pandas as pd
import glob
import numpy as np
import random
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [92]:
df_sim = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/df_all_round_sim.csv')

df_sim['match_id'].unique()

team = "SRH"
df_sim[(df_sim['batting_team']==team)|(df_sim['bowling_team']==team)]['match_id'].unique()

In [93]:
batter = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/batter_stats.csv')
bowler = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/bowler_stats.csv')

In [94]:
# runs * 0.5
# balls/dismissal * 0.5
# SR * 0.5
# fours * 0.25 + sixes * 0.5
# dot_% * (-2)
# 1/bpb * 50
# 

batter['points'] = batter['runs']*0.5 + \
            np.where(batter['balls_per_dismissal']<1000, batter['balls_per_dismissal']*0.5, batter['balls']*0.5)\
                   + batter['strike_rate']*0.5+\
                   batter['fours']*0.25 + batter['sixes']*0.5 + batter['dot_%']*(-2) + \
                   np.where(batter['bpb']>0, 50/batter['bpb'], 0)



# wickets * 12.5
# balls/dismissal = 1/SR * 500
# 1/economy * 100
# fours * (-0.25) + sixes * (-0.5)
# dot_% * 2
# bpb * 5
# 

bowler['points'] = bowler['wkts']*12.5 + \
            np.where(bowler['strike_rate']<1000, 500/bowler['strike_rate'], 0)\
                   + 100/bowler['economy']+\
                   bowler['fours']*(-0.25) + bowler['sixes']*(-0.5) + bowler['dot_%']*2 + \
                   np.where(((bowler['bpb']>0) & (bowler['bpb']<1000)), bowler['bpb']*5, bowler['balls']*5)



batter_mod = batter.copy()
bowler_mod = bowler.copy()
batter_mod.rename(columns={'striker':'player','points':'points_bat'},inplace=True)
bowler_mod.rename(columns={'bowler':'player','points':'points_ball'},inplace=True)

batter_mod = batter_mod[['player','points_bat']]
bowler_mod = bowler_mod[['player','points_ball']]

player_stats = batter_mod.merge(bowler_mod, on='player',how='outer')
player_stats.fillna(0, inplace=True)
player_stats['total_points'] = player_stats['points_bat']+player_stats['points_ball']

bat_team = df_sim[['striker','batting_team']].drop_duplicates()#.to_dict()
bowl_team = df_sim[['bowler','bowling_team']].drop_duplicates()

bat_team.columns = ['player','team']
bowl_team.columns = ['player','team']

team_info = bat_team.merge(bowl_team, on=['player','team'], how='outer')


player_stats = player_stats.merge(team_info, on='player')
player_stats = player_stats[['player','team','points_bat','points_ball','total_points']]

#player_stats.sample(4)

In [95]:
player_stats.sort_values('total_points',ascending=False).head(30)

,player,team,points_bat,points_ball,total_points
93,M Siddharth,CSK,0.000000,354.605152,354.605152
54,Washington Sundar,PBKS,44.570455,300.298889,344.869343
28,AR Patel,CSK,78.000000,254.572700,332.572700
92,T Natarajan,CSK,0.000000,322.441979,322.441979
48,HH Pandya,MI,41.771176,273.232254,315.003431
43,Mohammed Siraj,MI,100.592222,195.025126,295.617348
8,Abhishek Sharma,MI,139.876667,155.000000,294.876667
46,HV Patel,PBKS,43.780909,219.084595,262.865504
29,Rashid Khan,GT,58.250000,193.321373,251.571373
71,M Markande,LSG,103.000000,144.560635,247.560635


In [98]:
player_stats#[player_stats['points_bat']>100]

,player,team,points_bat,points_ball,total_points
0,V Kohli,DC,147.570987,0.000000,147.570987
1,SA Yadav,DC,87.953810,0.000000,87.953810
2,Shubman Gill,LSG,133.042806,0.000000,133.042806
3,Ishan Kishan,KKR,122.842528,0.000000,122.842528
4,SV Samson,GT,85.309259,0.000000,85.309259
5,RD Gaikwad,RCB,137.056598,0.000000,137.056598
6,RA Tripathi,RR,132.075000,0.000000,132.075000
7,PD Salt,RCB,131.000012,0.000000,131.000012
8,Abhishek Sharma,MI,139.876667,155.000000,294.876667
9,AM Rahane,KKR,107.120909,0.000000,107.120909


In [97]:
#ground stats
df_g = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/ground_stats.csv')

#bowler stats
df_bo_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_inn1.csv')
df_bo_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_inn2.csv')

df_bo = pd.concat([df_bo_1,df_bo_2], axis=0)
df_bo.reset_index(drop=True, inplace=True)

#batter stats
df_ba_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_inn1.csv')
df_ba_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_inn2.csv')

df_ba = pd.concat([df_ba_1,df_ba_2], axis=0)
df_ba.reset_index(drop=True, inplace=True)

#h2h factor
h2h = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/h2h_stats.csv')

#form factor-batter
form_ba_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_form_inn1.csv')
form_ba_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/batter_stats_form_inn2.csv')

form_ba = pd.concat([form_ba_1,form_ba_2], axis=0)
form_ba.reset_index(drop=True, inplace=True)

#form factor-bowler
form_bo_1 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_form_inn1.csv')
form_bo_2 = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_form_inn2.csv')

form_bo = pd.concat([form_bo_1,form_bo_2], axis=0)
form_bo.reset_index(drop=True, inplace=True)

df_ba.sample(5)

In [21]:
df_bo_1.iloc[:,:7].mean().to_list()/df_bo_1.iloc[:,:7].mean().sum()

array([0.05387563, 0.38051413, 0.06198354, 0.0027045 , 0.10257493,
       0.05703466, 0.3413126 ])

In [89]:
np.abs(np.random.normal(0.17, 0.15))

0.09304165684144586